In [16]:
import sys

sys.path.append('../')
import meep as mp
import meep.materials as mt
import matplotlib.pyplot as plt
import numpy as np
import utilities as util
import h5py
import clipboard_and_style_sheet
from IPython.display import Video

clipboard_and_style_sheet.style_sheet()

Set up the geometry of the problem. One waveguide laid out in the x direction

In [2]:
wl_wvgd = 3.5
n_center_wvl = mt.LiNbO3.epsilon(1 / wl_wvgd)[2, 2]  # z polarization
w_wvgd = 0.5 * wl_wvgd / n_center_wvl  # width of the waveguide is half a wavelength wide

dpml = 1  # PML thickness

center_wvgd = mp.Vector3(0, 0, 0)  # where the waveguide is centered
sx = 16  # size of the cell in the x direction
sy = w_wvgd * 10  # size of the cell in y direction

create the geometric objects from the above

In [3]:
blk = mp.Block(size=mp.Vector3(mp.inf, w_wvgd, mp.inf), center=center_wvgd)
cell = mp.Vector3(sx, sy, 0)
PML = mp.PML(dpml)

set the appropriate media for the geometric objects

In [4]:
blk.material = mt.LiNbO3

create the geometry and boundary layers list

In [5]:
geometry = [blk]
boundary_layers = [PML]

Done with geometry, moving on to sources, initialize an empty list for sources and append sources as you go

In [6]:
Sources = []

create a gaussian source instance and place it at the front of the waveguide

In [7]:
wl_src = 3.5
src = mp.GaussianSource(wavelength=wl_src, width=5)
pt = mp.Vector3() + center_wvgd
source = mp.Source(src=src,
                   component=mp.Ez,
                   center=pt + mp.Vector3(0, 0.25 * w_wvgd),
                   size=mp.Vector3())
Sources.append(source)

Done with sources, initialize the simulation instance

In [8]:
sim = mp.Simulation(cell_size=cell,
                    geometry=geometry,
                    sources=Sources,
                    boundary_layers=boundary_layers,
                    resolution=30)
sim.use_output_directory('sim_output')

'sim_output'

symmetries to exploit

In [9]:
Sym = [mp.Mirror(direction=mp.X)]

sim.symmetries = Sym

Run!

In [19]:
f = plt.figure()
Animate = mp.Animate2D(sim, fields=mp.Ez, f=f, realtime=False, normalize=True)

# sim.run(mp.to_appended("ez", mp.at_every(0.6, mp.output_efield_z)),
#         mp.at_beginning(mp.output_epsilon()),
#         until_after_sources=mp.stop_when_fields_decayed(50, mp.Ez, pt, 1e-3))

sim.run(mp.at_every(0.5, Animate),
        mp.at_beginning(mp.output_epsilon()),
        until_after_sources=mp.stop_when_fields_decayed(50, mp.Ez, pt, 1e-3))

# Close the animator's working frame
plt.close()

-----------
Initializing structure...
Halving computational cell along direction x
time for choose_chunkdivision = 0.000424147 s
Working in 2D dimensions.
Computational cell is 16 x 4.06667 x 0 with resolution 30
     block, center = (0,0,0)
          size (1e+20,0.405549,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
time for set_epsilon = 0.0416319 s
lorentzian susceptibility: frequency=0.0490243, gamma=0
lorentzian susceptibility: frequency=3.87492, gamma=0
lorentzian susceptibility: frequency=6.98942, gamma=0
lorentzian susceptibility: frequency=0.0459025, gamma=0
lorentzian susceptibility: frequency=4.11206, gamma=0
lorentzian susceptibility: frequency=7.52923, gamma=0
-----------
Meep: using output directory 'sim_output'
creating output file "sim_output/eps-000000.00.h5"...


/home/peterchang/miniconda3/envs/idp/lib/python3.10/site-packages/meep/simulation.py:1344: RuntimeWarning: Note: your sources include frequencies outside the range of validity of the material models. This is fine as long as you eventually only look at outputs (fluxes, resonant modes, etc.) at valid frequencies.
  warnings.warn(warn_src, RuntimeWarning)


     block, center = (0,0,0)
          size (1e+20,0.405549,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
field decay(t = 50.016666666666666): 0.332340509104965 / 0.332340509104965 = 1.0
field decay(t = 100.01666666666667): 1.4567804532034142e-07 / 0.332340509104965 = 4.38339718840387e-07
Normalizing field data...
run 3 finished at t = 100.01666666666667 (6001 timesteps)


In [24]:
filename="sim_output/wvgd_simple.mp4"
Animate.to_mp4(5, filename)
Video(filename)

Generating MP4...
